In [ ]:
import tensorflow as tf

use_gdrive = False
retrain_models = False

if use_gdrive:
    from google.colab import drive

    # Directory in Google Drive were we will save data.
    BASE_FOLDER = '/content/drive/My Drive/Colab Notebooks/tfm/'
    # Just set to True if you want to retrain the models.
    retrain_models = False

    drive.mount('/content/drive/')
else:
    BASE_FOLDER = './'

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  print('GPU device {} not found'.format(device_name))
else:
  print('Found GPU at: {}'.format(device_name))

In [ ]:
import tensorflow as tf
from tensorflow.keras import applications
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import optimizers
from tensorflow.keras import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D, DepthwiseConv2D
from tensorflow.keras import backend as k
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping
from tensorflow.keras.models import load_model
import os
import pickle
from tensorflow.keras.models import model_from_json
import matplotlib.pyplot as plt

image_width, image_height= 256, 256

nb_train_samples= 11000
nb_validation_sample=2000
batch_size = 8

model = applications.MobileNetV2(weights= "imagenet", include_top=False, input_shape=(image_height, image_width,3))

x=model.layers[7].output
#take the first 5 layers of the model
x=Flatten()(x)
x=Dense(1024, activation="relu")(x)
x=Dropout(0.5)(x)
x=Dense(384, activation="relu")(x)
x=Dropout(0.5)(x)
x=Dense(96, activation="relu")(x)
x=Dropout(0.5)(x)
predictions = Dense(30, activation="softmax")(x)


model_final = Model(inputs=model.input, outputs=predictions)

#model_final = load_model("weights_Mobile_Net.h5")

model_final.compile(loss="categorical_crossentropy", optimizer=optimizers.Nadam(lr=0.00001), metrics=["accuracy"])

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True,
                                   fill_mode="nearest",
                                   width_shift_range=0.3,
                                   height_shift_range=0.3,
                                   rotation_range=30)

test_datagen = ImageDataGenerator(rescale = 1./255,
                                  horizontal_flip = True,
                                  fill_mode = "nearest",
                                  zoom_range = 0.3,
                                  width_shift_range = 0.3,
                                  height_shift_range=0.3,
                                  rotation_range=30)

training_set = train_datagen.flow_from_directory('./train', target_size = (256, 256), batch_size = 8, class_mode = 'categorical')
test_set = test_datagen.flow_from_directory('./validation', target_size = (256, 256), batch_size = 8, class_mode = 'categorical') 
model_final.fit(training_set, steps_per_epoch = 1000, epochs = 80, validation_data = test_set, validation_steps=1000)
print(model.summary())

# Save your weights and model.
model_json=model_final.to_json()

with open("model.json", "w") as json_file:
    json_file.write(model_json)
model_final.save_weights("weights_VGG.h5")
model_final.save("model_27.h5")
#model_final.predict(test_set, batch_size=batch_size)

json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("weights_VGG.h5",by_name=True)
print("Loaded model from disk")
 
# evaluate loaded model on test data
loaded_model.compile(loss='categorical_crossentropy', optimizer='nadam', metrics=['accuracy'])

#print(loaded_model.summary())
loaded_model.fit_generator(training_set, steps_per_epoch = 1000, epochs = 100, validation_data = test_set,validation_steps=1000)
#score = loaded_model.evaluate(training_set,test_set , verbose=0)

In [ ]:
# From https://github.com/godofpdog/MobileNetV3_keras/blob/master/src/MobileNet_V3.py
def __conv2d_block(_inputs, filters, kernel, strides, is_use_bias=False, padding='same', activation='RE', name=None):
    x = Conv2D(filters, kernel, strides= strides, padding=padding, use_bias=is_use_bias)(_inputs)
    x = BatchNormalization()(x)
    if activation == 'RE':
        x = ReLU(name=name)(x)
    elif activation == 'HS':
        x = Activation(Hswish, name=name)(x)
    else:
        raise NotImplementedError
    return x

def __depthwise_block(_inputs, kernel=(3, 3), strides=(1, 1), activation='RE', is_use_se=True, num_layers=0):
    x = DepthwiseConv2D(kernel_size=kernel, strides=strides, depth_multiplier=1, padding='same', use_bias=False)(_inputs)
    x = BatchNormalization()(x)
    if is_use_se:
        x = __se_block(x)
    if activation == 'RE':
        x = ReLU()(x)
    elif activation == 'HS':
        x = Activation(Hswish)(x)
    else:
        raise NotImplementedError
    return x

def __global_depthwise_block(_inputs):
    assert _inputs._keras_shape[1] == _inputs._keras_shape[2]
    kernel_size = _inputs._keras_shape[1]
    x = DepthwiseConv2D((kernel_size, kernel_size), strides=(1, 1), depth_multiplier=1, padding='valid', use_bias=False)(_inputs)
    return x

def __se_block(_inputs, ratio=4, pooling_type='avg'):
    filters = _inputs._keras_shape[-1]
    se_shape = (1, 1, filters)
    if pooling_type == 'avg':
        se = GlobalAveragePooling2D()(_inputs)
    elif pooling_type == 'depthwise':
        se = __global_depthwise_block(_inputs)
    else:
        raise NotImplementedError
    se = Reshape(se_shape)(se)
    se = Dense(filters // ratio, activation='relu', kernel_initializer='he_normal', use_bias=False)(se)
    se = Dense(filters, activation='hard_sigmoid', kernel_initializer='he_normal', use_bias=False)(se)
    return multiply([_inputs, se])

def __bottleneck_block(_inputs, out_dim, kernel, strides, expansion_dim, is_use_bias=False, shortcut=True, is_use_se=True, activation='RE', num_layers=0, *args):
    with tf.name_scope('bottleneck_block'):
        # ** to high dim 
        bottleneck_dim = expansion_dim

        # ** pointwise conv 
        x = __conv2d_block(_inputs, bottleneck_dim, kernel=(1, 1), strides=(1, 1), is_use_bias=is_use_bias, activation=activation)

        # ** depthwise conv
        x = __depthwise_block(x, kernel=kernel, strides=strides, is_use_se=is_use_se, activation=activation, num_layers=num_layers)

        # ** pointwise conv
        x = Conv2D(out_dim, (1, 1), strides=(1, 1), padding='same')(x)
        x = BatchNormalization()(x)

        if shortcut and strides == (1, 1):
            in_dim = K.int_shape(_inputs)[-1]
            if in_dim != out_dim:
                ins = Conv2D(out_dim, (1, 1), strides=(1, 1), padding='same')(_inputs)
                x = Add()([x, ins])
            else:
                x = Add()([x, _inputs])
    return x

In [ ]:
# From LYTNetV2.
def make_divisible(x, divisible_by=8):
    import numpy as np
    return int(np.ceil(x * 1. / divisible_by) * divisible_by)

In [ ]:
# Convolutional layers.
def conv_layer(x, filters, kernel_size):
    x = layers.Conv2D(filters, kernel_size, use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    #x = layers.Activation('relu')(x)
    x = layers.Activation(tfl.hard_swish)(x)
    return x

def bottleneck_layer(inputs, output, kernel_size, strides, expansion_dim):
    __bottleneck_block(inputs, output, kernel, strides, expansion_dim)

# Defining the model.
def create_model(height, width, depth):
    inputShape = (height, width, depth)
    x = layers.Input(shape=input_shape)
    x = conv_layer(x, 16, (3, 3))
    x = layers.MaxPooling2D()(x)

    config_list = [
        # k, exp, c,  se,     nl,  s,
        [3, 16,  16,  False, 'RE', 1],
        [3, 64,  24,  False, 'RE', 2],
        [3, 72,  24,  False, 'RE', 1],
        [5, 72,  40,  True,  'RE', 2],
        [5, 120, 40,  True,  'RE', 1],
        [3, 240, 80,  False, 'HS', 2],
        [3, 200, 80,  False, 'HS', 1],
        [3, 480, 112, True,  'HS', 1],
        [5, 672, 160, True,  'HS', 2],
        [5, 960, 160, True,  'HS', 1],
        [3, 960, 320, False,  'RE', 1]
    ]
    x = bottleneck_layer(x, 24, (3, 3), 1, 64)

In [ ]:
# From: https://machinelearningmastery.com/use-keras-deep-learning-models-scikit-learn-python

# MLP for Pima Indians Dataset with 10-fold cross validation via sklearn
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
import numpy

# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)
# load pima indians dataset
dataset = numpy.loadtxt("pima-indians-diabetes.csv", delimiter=",")
# split into input (X) and output (Y) variables
X = dataset[:,0:8]
Y = dataset[:,8]
# create model
model = KerasBatchClassifier(build_fn=create_model, epochs=150, batch_size=10, verbose=0)
# evaluate using 10-fold cross validation
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(model, X, Y, cv=kfold)
print(results.mean())

## MobileNet V2

In [ ]:
# MobileNet V2
from sklearn.model_selection import cross_val_score
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
import numpy as np
import matplotlib.pyplot as plt
import os

number_of_classes = 2
BASE_FOLDER = './'
retrain_models = False
seed = 12

np.random.seed(seed)

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True,
                                   fill_mode="nearest",
                                   width_shift_range=0.3,
                                   height_shift_range=0.3,
                                   rotation_range=30)

validation_datagen = ImageDataGenerator(rescale = 1./255,
                                  horizontal_flip = True,
                                  fill_mode = "nearest",
                                  zoom_range = 0.3,
                                  width_shift_range = 0.3,
                                  height_shift_range=0.3,
                                  rotation_range=30)

training_flow = train_datagen.flow_from_directory('./train', target_size = (128, 128), batch_size = 1, class_mode = 'categorical')
validation_flow = validation_datagen.flow_from_directory('./validation', target_size = (128, 128), batch_size = 1, class_mode = 'categorical')

X = []
Y = []
batch_index = 0
while batch_index <= training_flow.batch_index:
    data = training_flow.next()
    X.append(data[0][0])
    Y.append(data[1][0])
    batch_index = batch_index + 1
X = np.asarray(X)
Y = np.asarray(Y)

print(np.shape(X))
sample = X[0]
print(Y[0])

# show image
imgplot = plt.imshow(sample)
plt.show()

# Split dataset, 90% train and 10% test.
split_1 = int(0.9 * len(X))

x_train = X[:split_1]
y_train = Y[:split_1]
x_test = X[split_1:]
y_test = Y[split_1:]

def add_top_model(base_model, number_of_classes, print_summary=False):
    model = models.Sequential()
    model.add(base_model)
    model.add(layers.Flatten())
    model.add(layers.Dense(256, activation='relu'))
    model.add(layers.Dense(number_of_classes, activation='softmax'))
    if print_summary:
        model.summary()
    return model

# Create model.
model_name = 'mobilnet_v2'
base_model = MobileNetV2(input_shape=(128, 128, 3), include_top=False)
base_model.trainable = True
model = add_top_model(base_model, number_of_classes, False)
model.compile(metrics=['accuracy'], loss='mean_squared_error', optimizer='adam')

neural_network = KerasClassifier(build_fn=model, 
                                 epochs=150, 
                                 batch_size=10, 
                                 verbose=0)
# Evaluate neural network using three-fold cross-validation
#accuracies = cross_val_score(neural_network, X, Y, scoring = "accuracy", n_jobs = 1)
#mean = accuracies.mean()
#variance = accuracies.std()

# Save checkpoints to get the best weights from all epochs, same name for all, to keep the best checkpoint.
filename = BASE_FOLDER+model_name+'checkpoint.hdf5'
checkpoint = ModelCheckpoint(filename, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks = [checkpoint]

# Fit model.
if retrain_models or not os.path.isfile(BASE_FOLDER+model_name+'.h5'):
    model.fit(x_train, y_train, epochs = 150, validation_data = (x_test, y_test), callbacks=callbacks)
    # Save the model.
    model.save(BASE_FOLDER+model_name+'.h5')
else:
    # Load the model.
    model = load_model(BASE_FOLDER+model_name+'.h5')

In [ ]:
# Let's evaluate it.
loss, accuracy = model.evaluate(x_test, y_test)
print('loss {} accuracy {}'.format(loss, accuracy))

In [ ]:
# Make a prediction.
img_index = 0
img = X[img_index]
predictions = model.predict(np.array([img]))
prediction = np.argmax(predictions)

print('Predicted: ', predictions[0])
print('Real: ', Y[img_index])

## YOLO

In [ ]:
import dataset

# Transform Pascal VOC format dataset to TFRecord
origin_dir = '../datasets/yolo_training/PTL_Dataset_876x657'
output_path = 'dataset.tfrecord'
label_map_path = 'dataset_label_map.pbtxt'
dataset.pascal_voc_to_tfrecord(origin_dir, output_path, label_map_path)

In [ ]:
import matplotlib.pyplot as plt
import tensorflow as tf

# Create an input pipeline.
record_dataset_path = 'dataset.tfrecord'
dataset = tf.data.TFRecordDataset(record_dataset_path)
raw_example = next(iter(dataset))
parsed = tf.train.Example.FromString(raw_example.numpy())

feature = parsed.features.feature
print(feature)
raw_img = feature['image/encoded'].bytes_list.value[0]
img = tf.image.decode_png(raw_img)
plt.imshow(img)

In [3]:
import dataset
import numpy as np
import tensorflow as tf
import utils
from keras_yolo3.yolo3.model import preprocess_true_boxes

def get_yolo3_anchors():
    anchors = '10,13, 16,30, 33,23, 30,61, 62,45, 59,119, 116,90, 156,198, 373,326'
    anchors = [float(x) for x in anchors.split(',')]
    return np.array(anchors).reshape(-1, 2)

def get_classes():
    label_map_path = 'dataset_label_map.pbtxt'
    return dataset.get_classes(label_map_path)

def read_tfrecord(example, input_shape, anchors, num_classes):
    features = {
        'image/encoded': tf.io.FixedLenFeature([], tf.string),
        'image/object/bbox/xmin': tf.io.FixedLenFeature([], tf.float32),
        'image/object/bbox/ymin': tf.io.FixedLenFeature([], tf.float32),
        'image/object/bbox/xmax': tf.io.FixedLenFeature([], tf.float32),
        'image/object/bbox/ymax': tf.io.FixedLenFeature([], tf.float32),
        'image/object/class/label': tf.io.FixedLenFeature([], tf.int64),
        
    }
    # Parse the serialized data so we get a dict with our data.
    parsed_example = tf.io.parse_single_example(serialized=example, features=features)
    # Print features
    sess = tf.compat.v1.InteractiveSession()
    with sess.as_default():
        for name, tensor in parsed_example.items():
            print('{}: {}'.format(name, tensor.eval()))
    print(parsed_example)

    image_data = parsed_example['image/encoded']
    x_min = parsed_example['image/object/bbox/xmin']
    y_min = parsed_example['image/object/bbox/ymin']
    x_max = parsed_example['image/object/bbox/xmax']
    y_max = parsed_example['image/object/bbox/ymax']
    class_id = parsed_example['image/object/class/label']
    print(class_id.get_shape())
    if tf.executing_eagerly():
        print(class_id.numpy())
    else:
        print(class_id) 
        with tf.InteractiveSession() as sess:
            print(class_id.eval()) 
    batch_size = len(class_id)
    print('Gets here')
    image_data = []
    box_data = []
    for i in range(batch_size):
        annotation = [x_min[i], y_min[i], x_max[i], y_max[i], class_id[i]]
        image, box = utils.get_random_data(image_data, annotation, input_shape, random=True)
        image_data.append(image)
        box_data.append(box)
    image_data = np.array(image_data)
    box_data = np.array(box_data)
    y_true = preprocess_true_boxes(box_data, input_shape, anchors, num_classes)
    return [image_data, *y_true], np.zeros(batch_size)

def get_dataset():
    # Create an input pipeline.
    record_dataset_path = 'dataset.tfrecord'
    dataset = tf.data.TFRecordDataset(record_dataset_path)
    input_shape = (576, 768) # multiple of 32, hw
    anchors = get_yolo3_anchors()
    num_classes = len(get_classes())
    #dataset = dataset.shuffle(5000).batch(32)
    dataset = dataset.map(lambda x: read_tfrecord(x, input_shape, anchors, num_classes))
    return dataset

In [ ]:
# Load data.

from PIL import Image
import glob, os
import numpy as np
import matplotlib.pyplot as plt
import xml.etree.ElementTree as ET
import xml.dom.minidom

data_path = '../datasets/yolo_training/PTL_Dataset_876x657'

X = []
Y = []
labels = []
try:
    os.chdir(data_path)
except OSError:
    print('Could not change directory')
for file in glob.glob("*.xml"):
    root = ET.parse(file).getroot()
    image_file = root.findtext('filename')
    for label in root.findall('object'):
        name = label.findtext('name')
        if name not in labels:
            labels.append(name)
        xmin = int(label.find('bndbox').findtext('xmin'))
        ymin = int(label.find('bndbox').findtext('ymin'))
        xmax = int(label.find('bndbox').findtext('xmax'))
        ymax = int(label.find('bndbox').findtext('ymax'))
        img = Image.open(image_file)
        img = np.asarray(img).flatten().tolist()
        X.append(img)
        Y.append([xmin, ymin, xmax, ymax, labels.index(name)])
print('All loaded')

In [ ]:
print(np.shape(X[0]))
sample = X[0]
print(Y[0])

# show image
imgplot = plt.imshow(np.asarray(sample).reshape(657, 876, 3))
plt.show()

# Split dataset, 90% train and 10% test.
split_1 = int(0.9 * len(X))

x_train = X[:split_1]
y_train = Y[:split_1]
x_test = X[split_1:]
y_test = Y[split_1:]

In [10]:
# create a YOLOv3 Keras model and save it to file
# based on https://github.com/experiencor/keras-yolo3
import struct
import numpy as np
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import ZeroPadding2D
from tensorflow.keras.layers import UpSampling2D
from tensorflow.keras.layers import Add, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model
import os
import tensorflow as tf

def _conv_block(inp, convs, skip=True):
    x = inp
    count = 0
    for conv in convs:
        if count == (len(convs) - 2) and skip:
            skip_connection = x
        count += 1
        if conv['stride'] > 1: x = ZeroPadding2D(((1,0),(1,0)))(x) # peculiar padding as darknet prefer left and top
        x = Conv2D(conv['filter'],
                   conv['kernel'],
                   strides=conv['stride'],
                   padding='valid' if conv['stride'] > 1 else 'same', # peculiar padding as darknet prefer left and top
                   name='conv_' + str(conv['layer_idx']),
                   use_bias=False if conv['bnorm'] else True)(x)
        if conv['bnorm']: x = BatchNormalization(epsilon=0.001, name='bnorm_' + str(conv['layer_idx']))(x)
        if conv['leaky']: x = LeakyReLU(alpha=0.1, name='leaky_' + str(conv['layer_idx']))(x)
    return Add()([skip_connection, x]) if skip else x

def make_yolov3_model(input_shape = None):
    if input_shape:
        input_image = Input(shape=input_shape)
    else:
        input_image = Input(shape=(None, None, 3))
    # Layer  0 => 4
    x = _conv_block(input_image, [{'filter': 32, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 0},
                                  {'filter': 64, 'kernel': 3, 'stride': 2, 'bnorm': True, 'leaky': True, 'layer_idx': 1},
                                  {'filter': 32, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 2},
                                  {'filter': 64, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 3}])
    # Layer  5 => 8
    x = _conv_block(x, [{'filter': 128, 'kernel': 3, 'stride': 2, 'bnorm': True, 'leaky': True, 'layer_idx': 5},
                        {'filter':  64, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 6},
                        {'filter': 128, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 7}])
    # Layer  9 => 11
    x = _conv_block(x, [{'filter':  64, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 9},
                        {'filter': 128, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 10}])
    # Layer 12 => 15
    x = _conv_block(x, [{'filter': 256, 'kernel': 3, 'stride': 2, 'bnorm': True, 'leaky': True, 'layer_idx': 12},
                        {'filter': 128, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 13},
                        {'filter': 256, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 14}])
    # Layer 16 => 36
    for i in range(7):
        x = _conv_block(x, [{'filter': 128, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 16+i*3},
                            {'filter': 256, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 17+i*3}])
    skip_36 = x
    # Layer 37 => 40
    x = _conv_block(x, [{'filter': 512, 'kernel': 3, 'stride': 2, 'bnorm': True, 'leaky': True, 'layer_idx': 37},
                        {'filter': 256, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 38},
                        {'filter': 512, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 39}])
    # Layer 41 => 61
    for i in range(7):
        x = _conv_block(x, [{'filter': 256, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 41+i*3},
                            {'filter': 512, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 42+i*3}])
    skip_61 = x
    # Layer 62 => 65
    x = _conv_block(x, [{'filter': 1024, 'kernel': 3, 'stride': 2, 'bnorm': True, 'leaky': True, 'layer_idx': 62},
                        {'filter':  512, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 63},
                        {'filter': 1024, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 64}])
    # Layer 66 => 74
    for i in range(3):
        x = _conv_block(x, [{'filter':  512, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 66+i*3},
                            {'filter': 1024, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 67+i*3}])
    # Layer 75 => 79
    x = _conv_block(x, [{'filter':  512, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 75},
                        {'filter': 1024, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 76},
                        {'filter':  512, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 77},
                        {'filter': 1024, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 78},
                        {'filter':  512, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 79}], skip=False)
    # Layer 80 => 82
    yolo_82 = _conv_block(x, [{'filter': 1024, 'kernel': 3, 'stride': 1, 'bnorm': True,  'leaky': True,  'layer_idx': 80},
                              {'filter':  255, 'kernel': 1, 'stride': 1, 'bnorm': False, 'leaky': False, 'layer_idx': 81}], skip=False)
    # Layer 83 => 86
    x = _conv_block(x, [{'filter': 256, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 84}], skip=False)
    x = UpSampling2D(2)(x)
    x = Concatenate()([x, skip_61])
    # Layer 87 => 91
    x = _conv_block(x, [{'filter': 256, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 87},
                        {'filter': 512, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 88},
                        {'filter': 256, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 89},
                        {'filter': 512, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 90},
                        {'filter': 256, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 91}], skip=False)
    # Layer 92 => 94
    yolo_94 = _conv_block(x, [{'filter': 512, 'kernel': 3, 'stride': 1, 'bnorm': True,  'leaky': True,  'layer_idx': 92},
                              {'filter': 255, 'kernel': 1, 'stride': 1, 'bnorm': False, 'leaky': False, 'layer_idx': 93}], skip=False)
    # Layer 95 => 98
    x = _conv_block(x, [{'filter': 128, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True,   'layer_idx': 96}], skip=False)
    x = UpSampling2D(2)(x)
    x = Concatenate()([x, skip_36])
    # Layer 99 => 106
    yolo_106 = _conv_block(x, [{'filter': 128, 'kernel': 1, 'stride': 1, 'bnorm': True,  'leaky': True,  'layer_idx': 99},
                               {'filter': 256, 'kernel': 3, 'stride': 1, 'bnorm': True,  'leaky': True,  'layer_idx': 100},
                               {'filter': 128, 'kernel': 1, 'stride': 1, 'bnorm': True,  'leaky': True,  'layer_idx': 101},
                               {'filter': 256, 'kernel': 3, 'stride': 1, 'bnorm': True,  'leaky': True,  'layer_idx': 102},
                               {'filter': 128, 'kernel': 1, 'stride': 1, 'bnorm': True,  'leaky': True,  'layer_idx': 103},
                               {'filter': 256, 'kernel': 3, 'stride': 1, 'bnorm': True,  'leaky': True,  'layer_idx': 104},
                               {'filter': 255, 'kernel': 1, 'stride': 1, 'bnorm': False, 'leaky': False, 'layer_idx': 105}], skip=False)
    model = Model(input_image, [yolo_82, yolo_94, yolo_106])
    return model

BASE_FOLDER = './'
retrain_models = False
input_shape = (576, 768, 3)

# Define the model.
model_name = 'yolo_v3'
model = make_yolov3_model(input_shape)
model.compile(metrics=['accuracy'], loss='mean_squared_error', optimizer='adam')

# Save checkpoints to get the best weights from all epochs, same name for all, to keep the best checkpoint.
filename = BASE_FOLDER+model_name+'checkpoint.hdf5'
checkpoint = ModelCheckpoint(filename, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks = [checkpoint]

#def generator_from_array(x_train, y_train):
#    while 1:
#        for i in range(len(x_train)):
#            yield (x_train[i], [y_train[i], y_train[i], y_train[i]])

dataset = get_dataset()

# Fit model.
if retrain_models or not os.path.isfile(BASE_FOLDER+model_name+'.h5'):
    model.fit(dataset, epochs = 150, callbacks=callbacks)
    # Save the model.
    model.save(BASE_FOLDER+model_name+'.h5')
else:
    # Load the model.
    model = load_model(BASE_FOLDER+model_name+'.h5')

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, 576, 768, 3) 0                                            
__________________________________________________________________________________________________
conv_0 (Conv2D)                 (None, 576, 768, 32) 864         input_5[0][0]                    
__________________________________________________________________________________________________
bnorm_0 (BatchNormalization)    (None, 576, 768, 32) 128         conv_0[0][0]                     
__________________________________________________________________________________________________
leaky_0 (LeakyReLU)             (None, 576, 768, 32) 0           bnorm_0[0][0]                    
____________________________________________________________________________________________

InvalidArgumentError: in converted code:

    /tmp/xpython_6327/1080054382.py:70 None  *
        dataset = dataset.map(lambda x: read_tfrecord(x, input_shape, anchors, num_classes))
    /tmp/xpython_6327/1080054382.py:32 read_tfrecord  *
        print('{}: {}'.format(name, tensor.eval()))
    /home/eduard/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/framework/ops.py:790 eval
        return _eval_using_default_session(self, feed_dict, self.graph, session)
    /home/eduard/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/framework/ops.py:5312 _eval_using_default_session
        return session.run(tensors, feed_dict)
    /home/eduard/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/client/session.py:960 run
        run_metadata_ptr)
    /home/eduard/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/client/session.py:1183 _run
        feed_dict_tensor, options, run_metadata)
    /home/eduard/anaconda3/lib/python3.7/site-packages/tensorflow

In [4]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

<TensorSliceDataset shapes: (), types: tf.int32>
Dummy function


In [2]:
import tensorflow as tf
import ptvsd

def read_tfrecord(x):
    ptvsd.debug_this_thread()
    print('Dummy function')
    return x

dataset = tf.data.Dataset.from_tensor_slices([1, 2, 3])
print(dataset)
dataset = dataset.map(lambda x: read_tfrecord(x))

<TensorSliceDataset shapes: (), types: tf.int32>


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: local variable 'kill' referenced before assignment
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Dummy function


In [3]:
import tensorflow as tf

tf.config.experimental_run_functions_eagerly(True)

@tf.function
def f(x, y):
    print('Operate')
    return x ** 2 + y

x = tf.constant([2, 3])
y = tf.constant([3, -2])
f(x, y)


Operate


<tf.Tensor: shape=(2,), dtype=int32, numpy=array([7, 7], dtype=int32)>